In [1]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import time
import csv
import queue
from datetime import datetime

/Users/beni/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
model_path = "/Users/beni/Desktop/Climbing Technique Tracker/Climbing_Tracker/pose_landmarker_full.task"

In [ ]:
"""The code in this block is partly taken from the official MediaPipe documentation and can be found 
here: https://ai.google.dev/edge/mediapipe/solutions/vision/pose_landmarker/python#live-stream_1"""

BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
PoseLandmarkerResult = mp.tasks.vision.PoseLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

saved_results = queue.Queue()


def print_result(result: PoseLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    print('pose landmarker result: {}'.format(result))
    saved_results.put(result)
    saved_results.put(timestamp_ms)


options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM, #sets the running mode to live stream
    result_callback=print_result)


"""The following code is written by myself using the OpenCV and MediaPipe documentation for help which
can be found in the readme file"""

#check how many frames per second Qsense sensors output
frames_per_second = 10
time_between_frames = 1 / frames_per_second


with PoseLandmarker.create_from_options(options) as landmarker:
    #Use openCV VideoCapture to start capturing from the webcam
    start_time = time.time()

    capture = cv2.VideoCapture(0)

    #Create a loop to read the latest frame from the camera using VideoCaptureRead()
    while capture.isOpened():

        ret, frame = capture.read()
     
        if not ret:
            print("broken")
            break

        #Convert the frame receieved from OpenCV to a MediaPipe Image Object
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
      
        #landmarker.detect_async(mp_image, timestamp_ms = int(time.time() * 1000))
        #from datetime import datetime
        landmarker.detect_async(mp_image, int(datetime.now().timestamp() * 1000))


        #color = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        cv2.imshow('frame', frame) #first param is window name
        cv2.waitKey(10)

        """The following frame limiter was taken from the following online answer:
         https://www.quora.com/How-do-I-decrease-the-frames-per-second-in-OpenCV-python"""
        
        elapsed_time = time.time() - start_time 
        time_to_wait = time_between_frames - elapsed_time 
        if time_to_wait > 0: 
            time.sleep(time_to_wait)
    

In [ ]:
results = []

while not saved_results.empty():
    landmark = saved_results.get() #get pose_landmark first
    
    time_stamp = saved_results.get() #get timestamp (comes after each landmark in the queue)

    results.append([time_stamp, landmark]) #append intermediate array into main array so it can be writen into CSV (each record in one array)


print("Collected", len(results), "results")


"""CSV code adapted from the following website: https://www.geeksforgeeks.org/python/writing-csv-files-in-python/"""

with open('results.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(results)

Collected 222 results
[[1751779657301, PoseLandmarkerResult(pose_landmarks=[[NormalizedLandmark(x=0.642767071723938, y=0.6456596851348877, z=-1.0123450756072998, visibility=0.9989086389541626, presence=0.990497887134552), NormalizedLandmark(x=0.6662377119064331, y=0.5589352250099182, z=-0.9376463294029236, visibility=0.9981821775436401, presence=0.9827421307563782), NormalizedLandmark(x=0.6828213930130005, y=0.558957576751709, z=-0.9377607703208923, visibility=0.998522937297821, presence=0.9821287393569946), NormalizedLandmark(x=0.6982135772705078, y=0.5586054921150208, z=-0.9378536939620972, visibility=0.9975981116294861, presence=0.9786496758460999), NormalizedLandmark(x=0.6031895279884338, y=0.5638018250465393, z=-0.9697690606117249, visibility=0.9983951449394226, presence=0.9857169389724731), NormalizedLandmark(x=0.5795444846153259, y=0.5673641562461853, z=-0.969270646572113, visibility=0.9988652467727661, presence=0.9855999946594238), NormalizedLandmark(x=0.5572586059570312, y=0.5